# Methods to Sonify a Pulse

## Astronify

In [1]:
import numpy as np
from astropy.table import Table
from astronify.series import SoniSeries

down_rate  = 10

data       = np.load('Data/Burst.npy')
data       = np.mean(data, axis=1)
data       = (data - data.min()) / (data.max() - data.min())
data       = np.mean(data[:len(data) // down_rate * down_rate].reshape(-1, down_rate), axis=1)

data_table = Table({
    'time': np.arange(len(data)),
    'flux': data
})

data_soni = SoniSeries(data_table)
data_soni.note_spacing = 0.01
data_soni.sonify()
data_soni.write('Audio.wav')

## Interpolate

In [2]:
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt

wave_length = 10 # second
sample_rate = 48000
repeat_num  = 10

data        = np.load('Data/Burst.npy')
data        = np.mean(data, axis=1)
data        = np.tile(data, repeat_num)

time        = np.arange(len(data)) / (len(data) - 1) * wave_length
f           = interpolate.interp1d(time, data, kind='linear')
wave_time   = np.arange(0, wave_length, 1/sample_rate)
wave_raw    = f(wave_time)
# wave_raw    = ((wave_raw - np.min(wave_raw)) / (wave_raw.max() - wave_raw.min()) * 6e4 - 3e4).astype(np.int16)
wave_raw    = ((wave_raw - np.min(wave_raw)) / (wave_raw.max() - wave_raw.min()) * 6e4).astype(np.int16)

###  AM Carrier

In [3]:
import numpy as np
import matplotlib.pyplot as plt

wave_length         = 1 # second
sample_rate         = 48000
down_rate           = 100
data                = np.load('Data/Burst.npy')
data                = np.mean(data, axis=1)
data                = np.mean(data[:len(data) // down_rate * down_rate].reshape(-1, down_rate), axis=1)

def pulsar_sound_am(time_array, flux_array, wave_length, rate):
    
    sample_interval = 1 / rate
    wave_data       = np.zeros(int(rate * wave_length) + 1)
    wave_data_i     = 0
    fc              = 500
    
    time_length     = len(time_array)
    flux_length     = len(flux_array)
    period          = time_array[time_length-1]
    time_array      = np.append(time_array, period)
    flux_array      = np.append(flux_array, flux_array[0])
    
    for i in np.arange(0, wave_length, sample_interval):
        i_tmp       = (i * 1000) % period
        for j in range(time_length):
            if (i_tmp >= time_array[j]) and (i_tmp <= time_array[j + 1]):
                tmp_xl                 = i_tmp - time_array[j]
                tmp_xu                 = time_array[j+1] - i_tmp
                tmp_xl_d               = tmp_xu / (tmp_xl + tmp_xu)
                tmp_xu_d               = tmp_xl / (tmp_xl + tmp_xu)
                wdi                    = flux_array[j] * tmp_xl_d + flux_array[j+1] * tmp_xu_d
                wave_data[wave_data_i] = (1.0 * (wdi**4 * 1e-16 - 20)) * np.cos(2 * np.pi * fc * i)
        wave_data_i += 1
    return wave_data

wave_raw             = pulsar_sound_am(np.arange(len(data)), data, wave_length, sample_rate)
wave_raw    = ((wave_raw - np.min(wave_raw)) / (wave_raw.max() - wave_raw.min()) * 6e4 - 3e4).astype(np.int16)

### Convolution

In [4]:
import wave
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from scipy import integrate, signal

def read_wave(file):
    with wave.open(file, 'rb') as f:
        nchannels, sampwidth, framerate, nframes = f.getparams()[:4]
        str_data                                 = f.readframes(nframes)
    wave_data       = np.frombuffer(str_data, dtype=np.short)
    wave_data.shape = -1, nchannels
    return wave_data.T[0]

# 读取音频文件
# pulse      = read_wave('Audio.wav')
pulse      = wave_raw
sound      = read_wave('Instruments/vio.wav')

# 归一化乐器声音
sounds     = stats.binned_statistic(
    x      = np.arange(0, len(sound)), 
    values = sound, 
    bins   = len(sound) * 44100 / 48000
)[0]

L          = integrate.simps(sounds)
sound_norm = sounds / L

# 卷积
wave_raw   = (signal.convolve(pulse, sound_norm, mode='same') / 10).astype(np.int16)

## STFT

In [ ]:
import numpy as np
from scipy.signal import stft, istft
import matplotlib.pyplot as plt

data = np.load('Data/Burst.npy')

# wave_length        = 2 # second
# sample_rate        = 48000
# 
# for i in range(100):
#     if np.int64(wave_length * sample_rate / 4096) >> i == 0:
#         nperseg    = 2 ** i
#         break

a = istft(
    data, 
    fs             = 1.0, 
    window         = 'hann', 
    nperseg        = 8191, 
    noverlap       = 8, 
    nfft           = 4096 * 2, 
    input_onesided = True, 
    boundary       = True, 
    time_axis      = -2, 
    freq_axis      = -1
)

f, t, b = stft(
    a[1], 
    nfft           = 4096 * 2, 
    nperseg        = 8191, 
    noverlap       = 8
)

wave_raw = np.mean(a[1][:a[1].shape[0] // 100 * 100].reshape(-1, 100), axis=1)
wave_raw = (wave_raw / np.abs(wave_raw).max() * 3e4).astype(np.int16)

In [6]:
import wave
with wave.open('Audio.wav', 'wb') as f:
    params = (1, 2, 48000, 1, 'NONE', 'not compressed')
    f.setparams(params)
    f.writeframesraw(wave_raw)

### Mel Spectrogram

In [7]:
import numpy as np
import librosa, copy
import soundfile as sf
from scipy import signal

def del_burst(data, exposure_cut=25):
    h, w            = data.shape
    data           /= np.mean(data, axis=0)
    flatten_data    = np.sort(data.flatten())
    vmin, vmax      = flatten_data[int(h*w/exposure_cut)], flatten_data[int(h*w/exposure_cut*(exposure_cut-1))]
    data[data<vmin] = vmin
    data[data>vmax] = vmax
    data            = (data - data.min()) / (data.max() - data.min())
    return data

###### 消色散后 ######
data            = np.load('Data/Burst.npy')
data            = data.astype(np.float64)
data            = del_burst(data)
data            = data[1024: 1024+2048]
data            = np.mean(data.reshape(64, 32, 512, 8), axis=(1, 3))

###### 消色散前 ######
# data            = np.load('Data/RawBurst.npy')
# data            = np.mean(data.reshape(128, 22, 512, 8), axis=(1, 3))

In [8]:
def melspectrogram2wav(mel):
    # transpose
    mel    = mel.T
    # de-noramlize
    mel    = (np.clip(mel, 0, 1) * max_db) - max_db + ref_db
    # to amplitude
    mel    = np.power(10.0, mel * 0.05)
    m      = _mel_to_linear_matrix(sr, n_fft, n_mels)
    mag    = np.dot(m, mel)
    # wav reconstruction
    wav    = griffin_lim(mag)
    # de-preemphasis
    wav    = signal.lfilter([1], [1, -preemphasis], wav)
    # trim
    wav, _ = librosa.effects.trim(wav)
    return wav.astype(np.float32)

def spectrogram2wav(mag):
    # transpose
    mag    = mag.T
    # de-noramlize
    mag    = (np.clip(mag, 0, 1) * max_db) - max_db + ref_db
    # to amplitude
    mag    = np.power(10.0, mag * 0.05)
    # wav reconstruction
    wav    = griffin_lim(mag)
    # de-preemphasis
    wav    = signal.lfilter([1], [1, -preemphasis], wav)
    # trim
    wav, _ = librosa.effects.trim(wav)
    return wav.astype(np.float32)

def _mel_to_linear_matrix(sr, n_fft, n_mels):
    m      = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=n_mels)
    m_t    = np.transpose(m)
    p      = np.matmul(m, m_t)
    d      = [1.0 / x if np.abs(x) > 1.0e-8 else x for x in np.sum(p, axis=0)]
    return np.matmul(m_t, np.diag(d))

def griffin_lim(spectrogram):
    X_best     = copy.deepcopy(spectrogram)
    for i in range(n_iter):
        X_t    = invert_spectrogram(X_best)
        est    = librosa.stft(X_t, n_fft=n_fft, hop_length=hop_length, win_length=win_length)
        phase  = est / np.maximum(1e-8, np.abs(est))
        X_best = spectrogram * phase
    X_t        = invert_spectrogram(X_best)
    y          = np.real(X_t)
    return y

def invert_spectrogram(spectrogram):
    return librosa.istft(spectrogram, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window='hann')

In [9]:
sr           = 48000  # Sample rate.
n_fft        = 4096   # fft points (samples)
frame_length = 0.04   # seconds
win_length   = int(sr * frame_length) # samples.
hop_length   = win_length // 4        # samples.

n_mels       = 512  # Number of Mel banks to generate
n_iter       = 32   # Number of inversion iterations
preemphasis  = 0.97 # or None
max_db       = 100
ref_db       = 20
top_db       = 15
a            = melspectrogram2wav(data)

sf.write('Audio.wav', a, sr, subtype='PCM_24')